In [12]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma



In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [15]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [16]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [17]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   index=False,
                                   header=False)


In [19]:
text_splitter = CharacterTextSplitter(
    chunk_size=10_000,  # big enough for your doc
    chunk_overlap=0,
    separator="\n"
)
documents = text_splitter.split_documents(raw_documents)


In [20]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [24]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\UMAIMA MEHMOOD\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\UMAIMA MEHMOOD\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory="chroma_db"
)

print("✅ Chroma DB created with HuggingFace embeddings!")


✅ Chroma DB created with HuggingFace embeddings!


In [26]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780067575208 First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson views as essential to life. In her insightfu

In [29]:
# Extract and clean the ISBN from the first search result
isbn_str = docs[0].page_content.split()[0].strip().replace('"', '').replace("'", "")
isbn = int(isbn_str)

# Retrieve the matching book
books[books["isbn13"] == isbn]


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...


In [30]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [31]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
86,9780060000141,0060000147,Poppy's Return,Avi,Juvenile Fiction,http://books.google.com/books/content?id=XbcMJ...,"There's trouble at Gray House, the girlhood ho...",2006.0,3.99,256.0,1086.0,Poppy's Return,"9780060000141 There's trouble at Gray House, t..."
260,9780060852559,0060852550,"Animal, Vegetable, Miracle",Barbara Kingsolver;Camille Kingsolver;Steven L...,Biography & Autobiography,http://books.google.com/books/content?id=qLkEY...,Bestselling author Barbara Kingsolver returns ...,2007.0,4.04,370.0,86130.0,"Animal, Vegetable, Miracle: A Year of Food Life",9780060852559 Bestselling author Barbara Kings...
274,9780060887964,0060887966,The Alchemist - Gift Edition,Paulo Coelho,Fiction,http://books.google.com/books/content?id=h6QuJ...,"""My heart is afraid that it will have to suffe...",2006.0,3.85,192.0,814.0,The Alchemist - Gift Edition,"9780060887964 ""My heart is afraid that it will..."
306,9780060932664,006093266X,Collected Novellas,Gabriel Garcia Marquez,Fiction,http://books.google.com/books/content?id=JRcVu...,"Renowned as a master of magical realism, Gabri...",1999.0,4.01,288.0,822.0,Collected Novellas,9780060932664 Renowned as a master of magical ...
316,9780060938109,0060938102,The Schopenhauer Cure,Irvin Yalom,Fiction,http://books.google.com/books/content?id=aDG4l...,Suddenly confronted with his own mortality aft...,2006.0,4.21,358.0,8663.0,The Schopenhauer Cure: A Novel,9780060938109 Suddenly confronted with his own...
332,9780060976095,0060976098,Fiskadoro,Denis Johnson,Fiction,http://books.google.com/books/content?id=YSTe7...,"Hailed by the New York Times as ""wildly ambiti...",1995.0,3.54,221.0,1463.0,Fiskadoro,9780060976095 Hailed by the New York Times as ...
391,9780061205699,0061205699,To Kill a Mockingbird (slipcased edition),Harper Lee,Fiction,http://books.google.com/books/content?id=M9lKH...,"At the age of eight, Scout Finch is an entrenc...",2006.0,4.27,323.0,250.0,To Kill a Mockingbird (slipcased edition),"9780061205699 At the age of eight, Scout Finch..."
430,9780064435260,0064435261,A Little Prairie House,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=pRSju...,"Long, long ago, a little girl named Laura Inga...",1999.0,4.19,32.0,1533.0,A Little Prairie House,"9780064435260 Long, long ago, a little girl na..."
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...
690,9780140447897,014044789X,Metamorphosis,Ovid;David Raeburn,Fiction,http://books.google.com/books/content?id=D3McX...,A new translation in hexameter verse of Ovid's...,2004.0,4.05,723.0,46550.0,Metamorphosis,9780140447897 A new translation in hexameter v...


In [32]:
import sys
print(sys.executable)


c:\ProgramData\anaconda3\python.exe


In [23]:
pip install sentence-transformers


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.6 MB 12.2 MB/s eta 0:00:01
   --------- ------------------------------ 2.9/11.6 MB 7.7 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.6 MB 5.5 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/11.6 MB 5.1 MB/s eta 0:00:02
   --------------- ------------------------ 4.5/11.6 MB 4.4 MB/s eta 0:00:02
   ----------------- ---------------------- 5.0/11.6 MB 3.9 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.6 MB 3.6 MB/s eta 0:00:02
   ------------------- -------------------- 5.8/11.6 MB 3.4 MB/s eta 0:00:02
   --------------------- ------------------ 6.3/11.6 MB 3.2 MB/s eta 0:00:02
   ---------------------- ----------------- 6.6/11.6 MB 3.1 MB/s eta 0:00:02
   ------------------------ --------------- 7.1/11.6 MB 3.0 MB/s eta 0:00:02
   ---